# Lecture 2 - Functional Programming

### 📕 Today's Agenda
---
 * Static methods and class attributes
 * Object copy
 * Iterators
 * Generators
 * Lambda functions
 * List comprehension

### 🧪 Theory
---
### Static classes and class attributes
**Class attributes definition**

Static attributes are sharing the value with all instances. They are inherited by all child classes.
Static attributes can be accesses also as instance attribute only if a instance attribute with same name does not exist,
otherwise the instance attribute will be accessed.

In [24]:
class Connection:
    default_port = 9000 # define and assign value for static attribute

class Car:
    cars_created = 0 # define and assign value for static attribute

    def __init__(self):
        Car.cars_created += 1 # set value for instance attribute
        #self.cars_created = 888 # instance attributes have priority

class Volvo(Car):

    def __init__(self):
        super().__init__()
        self.x = 1

v = Volvo()
print(v.cars_created) # print instance attribute
print(dir(v))
v.cars_created = 4 # define and set instance attribute
print(v.cars_created)
print('Static value', Car.cars_created) # print class attribute
vv = Volvo() # create new Car
print(v.cars_created) # print instance attribute
print(Car.cars_created) # print class attribute

1
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'cars_created', 'x']
4
Static value 1
4
2


**Static methods**

A static method is a method defined in a class but with no access to instance attributes or methods.
Static methods can be used to compute data based on static attributes.
When a class contains only static methods it is considered as namespace.

In [26]:
class Math:
    pi = 3.14
    # define static method using @staticmethod decorator
    @staticmethod
    def pow(a, b): # self argument is no longer needed
        return a ** b

    @staticmethod
    def mean(seq):
        return sum(seq) / len(seq)

    @staticmethod
    def circle_area(r):
        return 2 * Math.pi * r # use static attributes

print(Math.pow(2, 5)) # call a static method
print(Math.circle_area(3))

32
18.84


**Object copy**

The problem: you can not copy an object just by using equal sign.

In [30]:
volvo_v60 = Volvo()
volvo_v61 = volvo_v60
print('Are the same object?', volvo_v60 is volvo_v61)
print('ID volvo_v60', id(volvo_v60))
print('ID volvo_v61', id(volvo_v61))

Are the same object? True
ID volvo_v60 2195358368384
ID volvo_v61 2195358368384


The solution 😊

In [31]:
from copy import copy
volvo_v70 = copy(volvo_v60)
print('Are the same object?', volvo_v60 is volvo_v70)
print('ID volvo_v60', id(volvo_v60))
print('ID volvo_v70', id(volvo_v70))

Are the same object? False
ID volvo_v60 2195358368384
ID volvo_v61 2195352290976


Compounded objects issue

In [36]:
class User:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

class Notebook:
    def __init__(self, title: str, owner: User):
        self.title = title
        self.owner = owner

notebook = Notebook('Tests', User('Mihai', 'Dinu'))
notebook_copy = copy(notebook)

print('Original ID', id(notebook))
print('Copy ID', id(notebook_copy))
print("-" * 32)
print('Original User ID', id(notebook.owner))
print('Copy User ID', id(notebook_copy.owner))
print('Are them the same object?', 'Yes' if notebook_copy.owner is notebook.owner else 'No')

Original ID 2195352327696
Copy ID 2195352329568
--------------------------------
Original User ID 2195358517040
Copy User ID 2195358517040
Are they the same object? Yes


Solution? Deep Copy

In [37]:
from copy import deepcopy
notebook_copy2 = deepcopy(notebook)
print('Original ID', id(notebook))
print('Copy ID', id(notebook_copy2))
print("-" * 32)
print('Original User ID', id(notebook.owner))
print('Copy User ID', id(notebook_copy2.owner))
print('Are them the same object?', 'Yes' if notebook_copy2.owner is notebook.owner else 'No')

Original ID 2195352327696
Copy ID 2195345405600
--------------------------------
Original User ID 2195358517040
Copy User ID 2195345406272
Are them the same object? No


In order to change the behaviour for copy and deepcopy functions for a custom class, the __copy__ and __deepcopy__ methods
will be overwritten. This is used when you want to copy just some of attributes. Both magic methods must return a new object.

**Iterators**

An **iterable** object is an object that implements \_\_iter\_\_, which is expected to return an **iterator** object.

An iterator is an object that implements next, which is expected to return the next element of the iterable object that
returned it, and raise a *StopIteration* exception when no more elements are available.

In [18]:
# basic iterable class

import random

class DummyIter:
    pass

class TenRandInt:
    """This class will return an iterator that returns 10 random integers."""
    def __init__(self):
        self.count = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.count < 10:
            self.count += 1
            return random.randint(0, 10000)
        else:
            raise StopIteration # if already generated 10 ints, raise StopIteration in order to break the loop

# normal class instance
rand_ints = TenRandInt()

# get iterator object for rand_ints object
rand_ints_iter = iter(rand_ints)
print('Iterable object:', type(rand_ints), id(rand_ints))
print('Iterator object:', type(rand_ints_iter), id(rand_ints))

# check if an object can be iterated
try:
    print(iter(DummyIter()))
except TypeError as err:
    print('[ERROR] ', err)

print('DummyIter is iterable? ', hasattr(DummyIter(), '__iter__'))
print('TenRandInt is iterable? ', hasattr(TenRandInt(), '__iter__'))

print('DummyIter is an iterator? ', hasattr(DummyIter(), '__next__'))
print('TenRandInt is an iterator? ', hasattr(TenRandInt(), '__next__'))

# obtain next value from an iterator
print('\nValue returned by iterator:')
print(next(rand_ints))

# iterate with for through iterator elements
print('\nAll iterator values:')
for i in rand_ints:
    print(i)

Iterable object: <class '__main__.TenRandInt'> 1323680790800
Iterator object: <class '__main__.TenRandInt'> 1323680790800
[ERROR]  'DummyIter' object is not iterable
DummyIter is iterable?  False
TenRandInt is iterable?  True
DummyIter is an iterator?  False
TenRandInt is an iterator?  True

Value returned by iterator:
737

All iterator values:
9869
2027
2549
6790
3721
7041
7963
3873
7973


### 👩‍💻 Practice
---
1. Write a class that keeps track of its objects in a static list.
2. Write a class which overwrites the __copy__ method so you can check if an object has copies.
```python
shop = Shop()
shop_copy = copy(shop)
shop.has_copies() -> bool
```
3. Write a class that represents english alphabet and make it iterable so you can do *for* over this class and get each alphabet letter.

### 🏠 Homework
---